In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
%%time
from constants import DIR
data = np.load(f'{DIR}/data{8}/test/myopic_{10}_{3}.npz')
x, y = data['x'], data['y']
x.shape, y.shape

CPU times: user 59.1 ms, sys: 13.6 ms, total: 72.7 ms
Wall time: 95.3 ms


((19682, 4, 8, 8), (19682,))

In [31]:
names = ['map_size','student', 'Teacher', 'Train_Size', 'test', 'seed', 'Reward']
df = pd.read_csv('output/res_curr.txt', names=names)
print(df.shape)
df2 = df[(df.map_size==12)].groupby(['student']).Reward.mean().reset_index()
df2

(4971, 7)


,student,Reward
0,1,-0.496446
1,2,-0.127040
2,3,0.348052
3,4,0.597201
4,5,0.645543
5,6,0.708537
6,7,0.751936
7,8,0.832525
8,9,0.765268
9,10,0.825199


In [ ]:
df2 = df[(df.map_size==12) & (df.student==0)]
df2.loc[:,'student'] = df2['student'].astype(str)
df2.loc[:,'Train_Size'] = df2['Train_Size'].astype(str)
ax = sns.lineplot(data=df2, x='Teacher', y='Reward', hue='Train_Size', errorbar='se')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [12]:
names = ['map_size', 'Teacher', 'train_size', 'seed', 'res']
df = pd.read_csv('output/eval_acc.txt', names=names)
df.groupby(['train_size','map_size', 'Teacher']).res.mean()

train_size  map_size  Teacher
1024000     8         1          0.880222
                      2          0.850190
                      3          0.843364
                      4          0.814213
                      5          0.816517
                      6          0.819679
                      7          0.826760
                      8          0.835310
                      9          0.846141
                      10         0.851518
                      11         0.856180
                      12         0.856931
                      13         0.856669
                      14         0.854994
                      15         0.856212
                      16         0.855326
                      17         0.855784
                      18         0.856315
                      19         0.854309
                      20         0.854788
Name: res, dtype: float64

In [ ]:
# New
8,0,11,1000000,1000,0,1.737063038040251
8,0,7,1000000,1000,0,1.8574064630218068
8,0,8,1000000,1000,0,1.8212922538462877
8,0,16,1000000,1000,0,1.6762998601976065
8,0,13,1000000,1000,0,1.6745948742787191
8,0,9,1000000,1000,0,1.7977901335753728
8,0,12,1000000,1000,0,1.727754811599537
8,0,20,1000000,1000,0,1.6105042426305554
8,0,10,1000000,1000,0,1.7556572049105612
8,0,19,1000000,1000,0,1.6260097560881182
8,0,17,1000000,1000,0,1.6469156403898988
8,0,5,1000000,1000,0,1.8395065830360828
8,0,6,1000000,1000,0,1.888425513077001
8,0,15,1000000,1000,0,1.6817530591642733
8,0,18,1000000,1000,0,1.6218751254124977
8,0,4,1000000,1000,0,1.7656588591305082
8,0,14,1000000,1000,0,1.7006141977244933
8,0,1,1000000,1000,0,0.9120738847942531
8,0,3,1000000,1000,0,1.6278528175774505
8,0,2,1000000,1000,0,1.3724574572144252

In [ ]:
# Old
8,0,16,1000000,1000,0,4.216864691647996
8,0,17,1000000,1000,0,4.155643263936201
8,0,20,1000000,1000,0,4.275567572114419
8,0,14,1000000,1000,0,4.141503272168476
8,0,13,1000000,1000,0,3.9351915302048766
8,0,15,1000000,1000,0,4.1002358997320965
8,0,19,1000000,1000,0,4.350660008197166
8,0,18,1000000,1000,0,4.375860327525279
8,0,11,1000000,1000,0,3.916182925754724
8,0,12,1000000,1000,0,4.000723240068092
8,0,10,1000000,1000,0,3.721806339431124
8,0,9,1000000,1000,0,3.714816263050857
8,0,8,1000000,1000,0,3.5737149366454966
8,0,7,1000000,1000,0,3.3823733467993518
8,0,6,1000000,1000,0,3.1678324927441373
8,0,5,1000000,1000,0,3.0134143065412244
8,0,1,1000000,1000,0,1.7609404212506807
8,0,4,1000000,1000,0,2.8416005882425424
8,0,2,1000000,1000,0,2.4154991524870133
8,0,3,1000000,1000,0,2.611839614846001


In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from tqdm.contrib.concurrent import process_map
from model_reward import model_reward
import numpy as np
import pandas as pd
import seaborn as sns

np.set_printoptions(linewidth=1000, suppress=True)    
from constants import DIR


In [3]:
def geti(x, start):
    i0, i1 = start, start
    for i1 in range(i0, -2, -1):
        if (x[i0][3]!=x[i1][3]).mean():
            break
    i0 = i1+1
    for i1 in range(i0, len(x)+1):
        if i1==len(x):
            break
        if (x[i0][3]!=x[i1][3]).mean():
            break

    return i0, i1

In [48]:
def reward_eval(teacher):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    map_size = 8
    skip = map_size**2
    test_size = 1000 * skip
    
    x, y = [], []
    for test_group in range(test_size//3000+1):
        data = np.load(f'{DIR}/data{map_size}/test/myopic_{teacher}_{test_group}.npz')
        x.append(data['x'])   
        y.append(data['y'])
    x = np.concatenate(x)
    y = np.concatenate(y)
        
    avg_trues, avg_preds, avg_accs = [], [], []
    for seed in range(5):
        model = tf.keras.models.load_model(f'{DIR}/models{map_size}/{0}_{teacher}_{2048000}_{seed}.keras')
        for start in range(skip,test_size,skip):
            i0, i1 = geti(x, start)
            y_pred = model(x[i0:i1]).numpy().argmax(axis=1)            
            avg_preds.append(model_reward(map_size, y_pred, x[i0], gamma=1))
            avg_trues.append(model_reward(map_size, y[i0:i1], x[i0], gamma=1))
            avg_accs.append(np.mean(y_pred==y[i0:i1]))

    return avg_trues, avg_preds, avg_accs

In [49]:
# %lprun -f reward_eval reward_eval(10)

NameError: name 'x' is not defined

In [50]:
teachers = range(1,21)
res = process_map(reward_eval, teachers)

  0%|          | 0/20 [00:00<?, ?it/s]

In [51]:
print('Teacher     Data 1.0     Model 1.0     Model Acc')
for teacher, (at, ap, aa) in zip(teachers, res):
    print(f'     {teacher:>2}     {np.mean(at): 8.4f}      {np.mean(ap): 8.4f}      {np.mean(aa): 8.4f}')


Teacher     Data 1.0     Model 1.0     Model Acc
      1       1.1761        1.1679        0.9047
      2       2.4699        2.4796        0.8612
      3       3.8840        3.8666        0.8607
      4       5.3368        5.2785        0.8310
      5       6.6765        6.5915        0.8289
      6       7.8218        7.6809        0.8318
      7       8.7165        8.5355        0.8361
      8       9.2991        9.1461        0.8456
      9       9.6509        9.5118        0.8543
     10       9.8433        9.6870        0.8594
     11       9.9438        9.7356        0.8620
     12       9.9862        9.8153        0.8649
     13       9.9986        9.8184        0.8655
     14      10.0000        9.7855        0.8647
     15      10.0000        9.7866        0.8635
     16      10.0000        9.7669        0.8613
     17      10.0000        9.7823        0.8641
     18      10.0000        9.7976        0.8663
     19      10.0000        9.7940        0.8640
     20      10.0000

In [43]:
print('Teacher     Data 1.0     Model 1.0     Model Acc')
for teacher, (at, ap, aa) in zip(teachers, res):
    print(f'     {teacher:>2}     {np.mean(at): 8.4f}      {np.mean(ap): 8.4f}      {np.mean(aa): 8.4f}')


Teacher     Data 1.0     Model 1.0     Model Acc
      1       1.0053        0.9992        0.8777
      2       2.3761        2.3463        0.8480
      3       3.8015        3.7422        0.8413
      4       5.2124        5.1236        0.8119
      5       6.4454        6.4486        0.8103
      6       7.6752        7.5238        0.8110
      7       8.5810        8.4072        0.8207
      8       9.2031        9.0351        0.8316
      9       9.5350        9.4295        0.8369
     10       9.7611        9.5742        0.8435
     11       9.9246        9.6633        0.8524
     12       9.9792        9.6293        0.8500
     13       9.9952        9.7172        0.8474
     14      10.0000        9.6011        0.8465
     15      10.0000        9.7194        0.8489
     16      10.0000        9.6970        0.8490
     17      10.0000        9.6566        0.8513
     18      10.0000        9.7108        0.8532
     19      10.0000        9.6668        0.8457
     20      10.0000

In [90]:
DIR

'/storage1/fs1/chien-ju.ho/Active/gym/0_0_10_-1_0.1'

In [91]:
DIR = '/storage1/fs1/chien-ju.ho/Active/gym/0_0_10_-1_1'

map_size = 8
skip = map_size**2
test_size = 1000 * skip

teacher = 12

x, y = [], []
for test_group in range(test_size//3000+1):
    data = np.load(f'{DIR}/data{map_size}/test/myopic_{teacher}_{test_group}.npz')
    x.append(data['x'])   
    y.append(data['y'])
x = np.concatenate(x)
y = np.concatenate(y)


In [92]:
x.shape

(433004, 4, 8, 8)

In [94]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

model = tf.keras.models.load_model(f'{DIR}/models{map_size}/{0}_{teacher}_{1024000}_{0}.keras')

In [98]:
y_pred = model(x[:100])
(y_pred.numpy().argmax(axis=1) == y[:100])

array([ True, False,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, False, False,  True,  True,  True,  True, False, False,  True,  True,  True,  True,  True,  True, False,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, False,  True,  True, False,  True, False,  True,  True,  True,  True,  True,  True,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True, False,  True,  True,  True,  True, False,  True,  True,  True,  True,  True,  True,  True, False, False,  True, False, False,  True,  True,  True, False,  True,  True,  True,  True,  True,  True])

In [99]:
y_pred[1]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-2.475924 , -1.548105 ,  5.1766453,  6.6675086], dtype=float32)>

In [100]:
y[1]

2

In [101]:
x[1][0]+x[1][3]

array([[-0.4188975 , -0.94691545, -0.62580371, -0.33641625, -0.51967433, -0.9237623 , -0.0882566 , -0.90097753],
       [-0.52128694, -0.19114257, -0.6182928 , -0.39185908, -0.73693369, -0.80320842, -0.52767828, -0.56570602],
       [-0.11287209, -0.14506258, -0.56583204, -0.23888711, -0.52078922, -0.10246773, -0.33700212, -0.34544169],
       [-0.73675813, -0.12492105, -0.97728972, -0.87509282, -0.92938063, -0.22849997, -0.17014459, -0.39194057],
       [-0.35854481, -0.44998052, -0.71981049, -0.57439792, -0.56529242, -0.41275906, -0.4425546 ,  0.29931217],
       [-0.92139309, -0.82525196, -0.24258305, -0.92830323, -0.38645437, -0.82910307, -0.25328064, -0.92862349],
       [-0.71831009, -0.69796691, -0.39487655, -0.15172613, -0.78996438, -0.59254233, -0.8973052 , -0.71874989],
       [-0.62532694, -0.31032502, -0.72763664, -0.49800929, -0.98706822, -0.65217909, -0.02202295, -0.18107241]])